
# 🔍 Strategy Comparison for VLM-Based Recommendations

This notebook compares the following retrieval strategies:
- Basic image embedding
- Dual-mode image+text fusion
- Cosine reranking
- Personalized query fusion
- Metadata filtering

Metrics: Cosine distance, category match ratio, visual similarity (manual), etc.


In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from app.model import VLMEncoder
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import seaborn as sns

encoder = VLMEncoder()
df = pd.read_csv("../data/processed/catalog.csv")

# Sample image & prompt
img = Image.open("../data/raw/images/Folder/image1.jpg").convert("RGB")
prompt = "something like this but in black"
user_vec = np.random.rand(768)

vec_basic = encoder.encode(img)
vec_dual = encoder.encode_dual(img, prompt)
vec_personal = encoder.encode_personalized(img, prompt, user_vec)

# Embed dataset
catalog_vecs = []
categories = []
for _, row in df.iterrows():
    try:
        i = Image.open(f"../data/raw/images/{row['image_path']}").convert("RGB")
        v = encoder.encode_dual(i, row.get("description", ""))
        catalog_vecs.append(v)
        categories.append(row.get("category", "unknown"))
    except:
        continue

catalog_vecs = np.stack(catalog_vecs)

# Compute similarities
sim_basic = np.array([1 - cosine(vec_basic, v) for v in catalog_vecs])
sim_dual = np.array([1 - cosine(vec_dual, v) for v in catalog_vecs])
sim_personal = np.array([1 - cosine(vec_personal, v) for v in catalog_vecs])

# Top-k and stats
k = 5
def top_k_info(similarity, label):
    top_idx = np.argsort(similarity)[::-1][:k]
    top_scores = similarity[top_idx]
    top_cats = [categories[i] for i in top_idx]
    print(f"\n[{label}] Top-K Cosine Similarities:", top_scores)
    print(f"[{label}] Top-K Categories:", top_cats)
    return top_scores, top_cats

b_scores, b_cats = top_k_info(sim_basic, "Basic")
d_scores, d_cats = top_k_info(sim_dual, "Dual")
p_scores, p_cats = top_k_info(sim_personal, "Personalized")

# Plot score distribution
plt.figure(figsize=(10, 6))
sns.histplot(sim_basic, kde=True, label="Basic", color="blue")
sns.histplot(sim_dual, kde=True, label="Dual", color="green")
sns.histplot(sim_personal, kde=True, label="Personalized", color="red")
plt.legend()
plt.title("Similarity Score Distributions")
plt.xlabel("Cosine Similarity")
plt.ylabel("Frequency")
plt.show()
